In [46]:
import json
import os
import pandas as pd
import apache_beam as beam
import csv

# 0.경로 지정

### 0-1. Train path

In [47]:
file_list_path = '../202102n179/179.6개암 최소침습수술 AI학습데이터/01.데이터/1.Training/라벨링데이터/클래스2.(수술단계 동영상)/05.자궁내막암'

file_list = os.listdir(file_list_path)
list_total =[]
for i in file_list:
    file_name = os.listdir(file_list_path+"/"+i)
    for j in file_name:
        row_list=[]
        with open(file_list_path+"/"+i+"/"+j,"r")as f:
            data= json.load(f)
            row_list.append(data.get("metas").get("age"))
            row_list.append(data.get("metas").get("gender"))
            row_list.append(data.get("metas").get("operation"))
            row_list.append(data.get("metas").get("hospital"))
            row_list.append(data.get("metas").get("stage"))
            row_list.append(data.get("metas").get("grade"))
            row_list.append(data.get("metas").get("pathology"))
            row_list.append(data.get("phases").get("category_id"))
            
            list_total.append(row_list)

TRAIN = pd.DataFrame(list_total, columns = ['age','gender','operation','hospital','stage','grade','pathology', 'category_id']) 

TRAIN

,age,gender,operation,hospital,stage,grade,pathology,category_id
0,53,F,Endometrial cancer,가천대길병원,3,3,Non-endometrioid,36
1,53,F,Endometrial cancer,가천대길병원,3,3,Non-endometrioid,2
2,53,F,Endometrial cancer,가천대길병원,3,3,Non-endometrioid,34
3,53,F,Endometrial cancer,가천대길병원,3,3,Non-endometrioid,3
4,53,F,Endometrial cancer,가천대길병원,3,3,Non-endometrioid,4
...,...,...,...,...,...,...,...,...
1082,62,F,Endometrial cancer,서울대병원,1,1,Endometrioid,35
1083,62,F,Endometrial cancer,서울대병원,1,1,Endometrioid,33
1084,62,F,Endometrial cancer,서울대병원,1,1,Endometrioid,32
1085,62,F,Endometrial cancer,서울대병원,1,1,Endometrioid,5


### 0-2. VAL path

In [37]:
file_list_path_val= '../202102n179/179.6개암 최소침습수술 AI학습데이터/01.데이터/2.Validation/라벨링데이터/클래스2.(수술단계 동영상)/05.자궁내막암'

file_list_val= os.listdir(file_list_path_val)
list_total_val=[] 
for i in file_list_val:
    file_name_val= os.listdir(file_list_path_val+ "/" +i)
    for j in file_name_val:
        row_list_val =[]
        with open(file_list_path_val+"/"+i+"/"+j,"r") as f:
            data_val=json.load(f)
            row_list_val.append(data_val.get("metas").get("age"))
            row_list_val.append(data_val.get("metas").get("gender"))
            row_list_val.append(data_val.get("metas").get("operation"))
            row_list_val.append(data_val.get("metas").get("hospital"))
            row_list_val.append(data_val.get("metas").get("stage"))
            row_list_val.append(data_val.get("metas").get("grade"))
            row_list_val.append(data_val.get("metas").get("pathology"))
            row_list_val.append(data_val.get("phases").get("category_id"))
            
            list_total_val.append(row_list_val)    
            
VAL = pd.DataFrame(list_total_val, columns = ['age','gender','operation','hospital','stage','grade','pathology', 'category_id'])
VAL

,age,gender,operation,hospital,stage,grade,pathology,category_id
0,45,F,Endometrial cancer,가천대길병원,1,1,Endometrioid,3
1,45,F,Endometrial cancer,가천대길병원,1,1,Endometrioid,6
2,45,F,Endometrial cancer,가천대길병원,1,1,Endometrioid,7
3,45,F,Endometrial cancer,가천대길병원,1,1,Endometrioid,8
4,45,F,Endometrial cancer,가천대길병원,1,1,Endometrioid,9
...,...,...,...,...,...,...,...,...
126,64,F,Endometrial cancer,서울대병원,1,3,Endometrioid,34
127,64,F,Endometrial cancer,서울대병원,1,3,Endometrioid,2
128,64,F,Endometrial cancer,서울대병원,1,3,Endometrioid,37
129,64,F,Endometrial cancer,서울대병원,1,3,Endometrioid,18


# process

# 1. statistic 생성 및 시각화

In [16]:
import sys, os
import tempfile, urllib, zipfile
import tensorflow_data_validation as tfdv
import tensorflow_metadata.proto.v0.statistics_pb2 as anomalies_pb2
import tensorflow_metadata.proto.v0.statistics_pb2 as schema_pb2
import tensorflow_metadata.proto.v0.statistics_pb2 as statistics_pb2
from tensorflow.python.lib.io import file_io
from tensorflow_data_validation.utils import slicing_util
from tensorflow_metadata.proto.v0.statistics_pb2 import DatasetFeatureStatisticsList, DatasetFeatureStatistics
from tensorflow_data_validation.anomalies.proto import validation_config_pb2

print('TFDV version: {}'.format(tfdv.version.__version__))

TFDV version: 1.3.0


### 1-1. statistic 생성  

In [38]:
[methods for methods in dir(tfdv) if "generate" in methods]
>>> ['generate_statistics_from_csv',
'generate_statistics_from_dataframe',
'generate_statistics_from_tfrecord']

['generate_statistics_from_csv',
 'generate_statistics_from_dataframe',
 'generate_statistics_from_tfrecord']

In [39]:
train_stats = tfdv.generate_statistics_from_dataframe(TRAIN)
# method : tfdv.generate_statistics_from_csv(data_location, column_names=None, delimiter=',', output_path=None, stats_options=<tenso

In [20]:
train_stats

datasets {
  num_examples: 1087
  features {
    type: STRING
    string_stats {
      common_stats {
        num_non_missing: 1087
        min_num_values: 1
        max_num_values: 1
        avg_num_values: 1.0
        num_values_histogram {
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 108.7
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 108.7
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 108.7
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 108.7
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 108.7
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 108.7
          }
          buckets {
            low_value: 

### 1-2. 시각화 

In [40]:
####여전히 시각화 안됨 일단 패스

tfdv.visualize_statistics(train_stats)

# method : tfdv.visualize_statistics(lhs_statistics, rhs_statistics=None, lhs_name='lhs_statistics', rhs_name='rhs_statistics)

In [44]:
#compute stats for evaluation data
eval_stats = tfdv.generate_statistics_from_dataframe(VAL)

### 2. 스키마 추론

#### 각 feature 별로 스키마를 추론함

In [41]:
schema = tfdv.infer_schema(statistics=train_stats)
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'age',INT,required,,-
'gender',STRING,required,,'gender'
'operation',STRING,required,,'operation'
'hospital',STRING,required,,'hospital'
'stage',INT,required,,-
'grade',INT,required,,-
'pathology',STRING,required,,'pathology'
'category_id',BYTES,required,,-


/opt/conda/lib/python3.6/site-packages/tensorflow_data_validation/utils/display_util.py:180: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'gender','F'
'operation','Endometrial cancer'
'hospital',"'가천대길병원', '서울대병원'"
'pathology',"'Endometrioid', 'Non-endometrioid'"


# 3. 데이터 검증

In [45]:
anomalies = tfdv.validate_statistics(statistics=eval_stats, schema=schema)
tfdv.display_anomalies(anomalies)

/opt/conda/lib/python3.6/site-packages/tensorflow_data_validation/utils/display_util.py:217: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)
